# Bayesian Personalized Ranking From Scratch

In [3]:
import os

path = os.getcwd()

In [4]:
%load_ext watermark
%load_ext autoreload
%autoreload 2

import sys
from itertools import islice
from math import ceil
from subprocess import call

import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, dok_matrix
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import normalize
from tqdm import trange

%watermark -a 'Ethen' -d -t -v -p numpy,pandas,scipy,sklearn,tqdm

Author: Ethen

Python implementation: CPython
Python version       : 3.9.7
IPython version      : 7.30.1

numpy  : 1.21.4
pandas : 1.3.5
scipy  : 1.7.3
sklearn: 1.0.1
tqdm   : 4.62.3



In [23]:
data_dir = "../data/"
file_dir = os.path.join(data_dir, "ml-100k")
file_path = os.path.join(data_dir, file_dir, "u.data")

if not os.path.isdir(file_dir):
    call(["curl", "-O", "http://files.grouplens.org/datasets/movielens/ml-100k.zip"])
    call(["unzip", "ml-100k.zip", "-d", "../data/"])
    call(["rm", "ml-100k.zip"])

names = ["user_id", "item_id", "rating", "timestamp"]
df = pd.read_csv(file_path, sep="\t", names=names)

print(df.shape)
df.head()

(100000, 4)


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [24]:
def create_matrix(data, users_col, items_col, ratings_col, threshold=None):
    """Create the sparse user-item interaction matrix,
    if the data is not in the format where the interaction only
    contains the positive items (indicated by 1), then use the
    threshold parameter to determine which items are considered positive

    Parameters
    ----------
    data : DataFrame
        implicit raging data

    users_col : str
        user column name

    items_col : str
        item column name

    ratings_col : str
        implicit rating column name

    threshold : int, default None
        threshold to determine whether th user-item pair is
        a positive feedback

    Returns
    -------
    ratings : scipy sparse csr_matrix, shape [n_users, n_items]
        user/item ratings martix

    data : DataFrame
        implicit rating data that retains only the positive feedback
        (if specified to do so)
    """
    if threshold is not None:
        data = data[data[ratings_col] >= threshold].copy()
        data[ratings_col] = 1

    for col in (items_col, users_col, ratings_col):
        data[col] = data[col].copy().astype("category")

    ratings = csr_matrix(
        (data[ratings_col], (data[users_col].cat.codes, data[items_col].cat.codes))
    )
    ratings.eliminate_zeros()
    return ratings, data

In [25]:
users_col = "user_id"
items_col = "item_id"
ratings_col = "rating"
threshold = 3
X, df = create_matrix(df, users_col, items_col, ratings_col, threshold)

X

<943x1574 sparse matrix of type '<class 'numpy.int64'>'
	with 82520 stored elements in Compressed Sparse Row format>

In [26]:
def create_train_test(ratings, test_size=0.2, seed=1234):
    """Split the user-item interaction matrix into train and test set
    by removing some of the interactions from every user and pretend
    that we never seen them

    Parameters
    ----------
    ratings : scipy sparse csr_matrix, shape [n_users, n_items]
        The user-item interactions matrix

    test_size : float between 0.0 and 1.0, default 0.2
        Proportion of the user-item interactions for each user
        in the dataset to move to the test set; e.g. if set to 0.2
        and a user has 10 interactions, then 2 will be moved to the
        test set

    seed : int, default 1234
        Seed for reproducible random splitting the
        data into train/test set

    Returns
    -------
    train : scipy sparse csr_matrix, shape [n_users, n_items]
        Training set

    test : scipy sparse csr_matrix, shape [n_users, n_items]
        Test set
    """
    assert test_size < 1.0 and test_size > 0.0

    # Dictionary Of Keys based sparse matrix is more efficient
    # for constructing sparse matries incrementally compared with csr_matrix
    train = ratings.copy().todok()
    test = dok_matrix(train.shape)

    # for all the users assign randomly chosen interactions
    # to the test and assign those interactions to zeoro in training;
    # when computing the interactions to go into the test set,
    # remenber to round up the numbers (e.g. a user has 4 ratings, if the
    # test_size is 0.2, the 0.8 ratings will go to test, thus we need to
    # round up to ensure the test set gets at least 1 rating)
    rstate = np.random.RandomState(seed)
    for u in range(ratings.shape[0]):
        split_index = ratings[u].indices
        n_splits = ceil(test_size * split_index.shape[0])
        test_index = rstate.choice(split_index, size=n_splits, replace=False)
        test[u, test_index] = ratings[u, test_index]
        train[u, test_index] = 0

    train, test = train.tocsr(), test.tocsr()
    return train, test

In [27]:
X_train, X_test = create_train_test(X, test_size=0.2, seed=1234)
X_train

<943x1574 sparse matrix of type '<class 'numpy.int64'>'
	with 65641 stored elements in Compressed Sparse Row format>

In [40]:
class BPR:
    """Baysian Personalized Ranking (BPR) for implicit feedback data

    Parameters
    ----------
    learning_rate : float, default 0.01
        learning rate for gradient descent

    n_factors : int, default 20
        Number/dimension of user and item latent factors

    batch_size : int, default 1000
        batch size for batch gradient descent, the original paper
        uses stochastic gradient descent (i.e., batch size of 1),
        but this can make the training unstable (very sensitive to
        learning rate)

    reg : int, default 0.01
        Regularization term for the user and item latent factors

    seed : int, default 1234
        Seed for the randomly initialized user, item latent factors

    verbose : bool, default True
        Whether to print progress bar while training

    Attributes
    ----------
    user_factors : 2d ndarray, shape [n_users, n_factors]
        User latent factors learnt

    item_factors : 2d ndarray, shape [n_items, n_factors]
        Item latent factors learnt

    References
    ----------
    S. Rendle, C. Freudenthaler, Z. Gantner, L. Schmidt-Thieme
    Bayesian Personalized Ranking from Implicit Feedback
    - https://arxiv.org/abs/1205.2618
    """

    def __init__(
        self,
        learning_rate=0.01,
        n_factors=15,
        n_iters=10,
        batch_size=1000,
        reg=0.01,
        seed=1234,
        verbose=True,
    ):
        self.reg = reg
        self.seed = seed
        self.verbose = verbose
        self.n_iters = n_iters
        self.n_factors = n_factors
        self.batch_size = batch_size
        self.learning_rate = learning_rate

        # to avoid re-computation at predict
        self._prediction = None

    def fit(self, ratings):
        """
        Parameters
        ----------
        ratings : scipy sparse csr_matrix, shape [n_users, n_items]
            sparse matrix of user-item interactions
        """
        indptr = ratings.indptr
        indices = ratings.indices
        n_users, n_items = ratings.shape

        # ensure batch size makes sense, since the algorithm involves
        # for each step randomly sample a user, thus the batch size
        # should be smaller than the total number of users or else
        # we would be sampling the user with replacement
        batch_size = self.batch_size
        if n_users < batch_size:
            batch_size = n_users
            sys.stderr.write(
                "WARNING: Batch size is greater than number of users,　"
                "swiching to a batch size of {}\n".format(n_users)
            )

        batch_iters = n_users // batch_size
        # Initialize random weights
        rstate = np.random.RandomState(self.seed)
        self.user_factors = rstate.normal(size=(n_users, self.n_factors))
        self.item_factors = rstate.normal(size=(n_items, self.n_factors))

        # progress bar for training iteration if verbose is turned on
        loop = range(self.n_iters)
        if self.verbose:
            loop = trange(self.n_iters, desc=self.__class__.__name__)

        for _ in loop:
            for _ in range(batch_iters):
                sampled = self._sample(n_users, n_items, indices, indptr)
                sampled_users, sampled_pos_items, sampled_neg_items = sampled
                self._update(sampled_users, sampled_pos_items, sampled_neg_items)

        return self

    def _sample(self, n_users, n_items, indices, indptr):
        """Sample batches of random triplets u, i, j"""
        sampled_pos_items = np.zeros(self.batch_size, dtype=np.int)
        sampled_neg_items = np.zeros(self.batch_size, dtype=np.int)
        sampled_users = np.random.choice(n_users, size=self.batch_size, replace=False)

        for idx, user in enumerate(sampled_users):
            pos_items = indices[indptr[user] : indptr[user + 1]]
            pos_item = np.random.choice(pos_items)
            neg_item = np.random.choice(n_items)
            while neg_item in pos_items:
                neg_item = np.random.choice(n_items)

            sampled_pos_items[idx] = pos_item
            sampled_neg_items[idx] = neg_item

        return sampled_users, sampled_pos_items, sampled_neg_items

    def _update(self, u, i, j):
        """Update according to the bootstrapped user u,
        positive item i and negative item j
        """
        user_u = self.user_facotrs[u]
        item_i = self.item_factors[i]
        item_j = self.item_factors[j]

        # decompose the estimator, compute the difference between
        # the score of the positive items and negative items; a
        # native implementation might look like the following:
        # r_ui = np.diag(user_u.dot(item_i.T))
        # r_uj = np.diag(user_u.dot(item_j.T))
        # r_uij = r_ui - r_uj

        # however, we can do better, so
        # for batch dot product, instead of doing the dot product
        # then only extract the diagonal element (which is the value
        # of that current batch), we perform a hadamard product,
        # i.e. matrix element-wise product then do a sum along the column will
        # be more efficient since it's less operations
        # http://poeple.revoledu.com/kardi/tutorial/LinearAlgebra/HadamarProduct.html
        # r_ui = np.sum(user_u * item_i, axis=1)
        #
        # then we can achieve another speedup by doing the difference
        # on the positive and negative item up front instead of computing
        # r_ui and r_uj separately, these two idea will speed up the operations
        # from 1:14 down to 0.36
        r_uij = np.sum(user_u * (item_i - item_j), axis=1)
        sigmoid = np.exp(-r_uij) / (1.0 + np.exp(-r_uij))
        
        # repeat the 1 dimension sigmoid n_factors times so 
        # the dimension will match when doing the update
        sigmoid_tiled = np.tile(sigmoid, (self.n_factors, 1)).T
        
        # update using gradient descent
        grad_u = sigmoid_tiled * (item_j - item_i) + self.reg * user_u

    def predict(self):
        pass

    def _predict_user(self, user):
        pass

    def recommend(self, ratings, N=5):
        pass

    def _recommend_user(self, ratings, user, N):
        pass

    def get_similar_items(self, N=5, item_ids=None):
        pass

In [42]:
# parameters were randomly chosen
bpr_params = {
    "reg": 0.01,
    "learning_rate": 0.1,
    "n_iters": 160,
    "n_factors": 15,
    "batch_size": 100,
}

bpr = BPR(**bpr_params)
bpr.fit(X_train)

BPR:   0%|                                                                                                                | 0/160 [00:00<?, ?it/s]/tmp/ipykernel_25/3819263579.py:107: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sampled_pos_items = np.zeros(self.batch_size, dtype=np.int)
/tmp/ipykernel_25/3819263579.py:108: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np

## Evaluation

## Item Recommendations

## Reference

- http://ethen8181.github.io/machine-learning/recsys/4_bpr.html